# Motor de Recuperación parte 3 : Reconocimiento de entidades.

---

**Bruno Santome: 100405889**

**David Valencia: 100386286**














In [155]:
from google.colab import drive
drive.mount ('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Imports y paths de documentación

---

En esta sección se incluyen todos las librerías usadas en el cuaderno y ciertas variables como los paths de cada documento necesario para el desarollo del motor de recuperación .


In [156]:
#Se añaden todos los imports, por comdidad.  
from google.colab import files
import os, nltk
from bs4 import BeautifulSoup
from pathlib import Path
from inspect import formatannotationrelativeto
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
import numpy as np
import threading
import re
from tabulate import tabulate
import math
#Se define aqui la variable para el proceso de stemming, para no repetir.
ps = PorterStemmer()
snow_stemmer = SnowballStemmer(language='english')
nltk.download('stopwords')
stopWords = set(stopwords.words('english'))

#Definimos la lista de cáracteres espciales que queremos quitar (ver si hay que quitar números)
special_characters=['@','#','$','*','&','/','[',']',
                    '.',',','-','?','!','(',')','|',';',':',
                    '"','“', '”','\n','\.',' .\ ','<','>','\t',"'",'°',
                    '�','©', '®', '¦','\\','!', '?','~','%','_',
                    '-', '+', '=', '^','‘','’','?','—' ]

##############################PATHS##################################

#Cambiar El path manualmente
#BRUNO PATH: "/content/drive/MyDrive/RAI/Practica_MOTOR/Motor_Recuperación_Evaluación/documents.biased"


###############CAMBIAR MANUALMENTE ESTAS VARIABLES.##################

#Esta variable es la que contiene todos los subdirectorios con sus respectivos documentos htmls. 
rootdir = "/content/drive/MyDrive/RAI/Practica_MOTOR/Motor_Recuperación_Evaluación/documents.biased"

#Esta variable es la que contiene el archivo XML donde se guardan las consultas a realizar.
fn_consultas = "/content/drive/MyDrive/RAI/Practica_MOTOR/Motor_Recuperación_Evaluación/topics.xml"

#Esta variable contiene el archivo con las relevancías de cada documento.
fn_relevancia = "/content/drive/MyDrive/RAI/Practica_MOTOR/Motor_Recuperación_Evaluación/union23.trel"

fn_nombres = "/content/drive/MyDrive/RAI/Practica_MOTOR/Motor_Recuperación_Evaluación/Person_names"

#####################################################################

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Recogida de texto en los documentos HTML

---

Los archivos html estan subidos al drive, hay un total de 99 subdirectorios y cada uno con sus respectivos htmls, tenemos que leerlos y almacenarlos en una variable para proceder al filtrado de etiquetas y otros. 

Se proporciona el codigo de  recogida lineal y el multiHilo. 
En concreto, el codigo lineal esta comentado ya que el multihilo da mejores tiempos. 

---


TIEMPO EJECUCION para leer los 1965 documentos: 9 mins 40' | 11 mins 45'
 ==> Ver si se puede mejorar. Tiempo max hasta ahora: 28 mins


In [157]:
#Codigo lineal
"""
Lista_Content =[]
Name_Files = []

for file in os.listdir(rootdir):
        d = os.path.join(rootdir, file)
        #print(d)
        if os.path.isdir(d):
            for file in os.listdir(d):
              if file.endswith('.html'):
                  Name_Files.append(file)
                  fn = os.path.join(d,file)
                  with open (fn, 'r') as fichero:
                      soup = BeautifulSoup(fichero.read(), 'html.parser')
                      for script in soup(["script", "style"]):
                          script.decompose()
                      #el método stripped_strings de la libreria beatifulSoup quita los espacios y los saltos de linea innecesarios              
                      Lista_Content.append(list(soup.stripped_strings)) #lo juntamos todo en una sola lista con toda el texto sin espacios de los cinco archivos html. 
                  
print(Lista_Content[0])
"""

'\nLista_Content =[]\nName_Files = []\n\nfor file in os.listdir(rootdir):\n        d = os.path.join(rootdir, file)\n        #print(d)\n        if os.path.isdir(d):\n            for file in os.listdir(d):\n              if file.endswith(\'.html\'):\n                  Name_Files.append(file)\n                  fn = os.path.join(d,file)\n                  with open (fn, \'r\') as fichero:\n                      soup = BeautifulSoup(fichero.read(), \'html.parser\')\n                      for script in soup(["script", "style"]):\n                          script.decompose()\n                      #el método stripped_strings de la libreria beatifulSoup quita los espacios y los saltos de linea innecesarios              \n                      Lista_Content.append(list(soup.stripped_strings)) #lo juntamos todo en una sola lista con toda el texto sin espacios de los cinco archivos html. \n                  \nprint(Lista_Content[0])\n'

El codigo de abajo es con threads, para mejorar el tiempo de ejecución

In [158]:
#Codigo con threads
Lista_Content =[]
Name_Files = []

#2 min 30 ' , 4min
def process_file(file_entry):
    Name_Files.append(file_entry.name)
    with open(file_entry.path, 'r') as f:   
        soup = BeautifulSoup(f.read(), 'html.parser')
        for script in soup(["script", "style"]): # le pasamos una lista de todos los tags que contengan script y style
            script.decompose() 
        Lista_Content.append(list(soup.stripped_strings))

threads = []
for dir_entry in os.scandir(rootdir):
    if dir_entry.is_dir() and not dir_entry.name.startswith('.'):
        for file_entry in os.scandir(dir_entry):
            if file_entry.is_file() and file_entry.name.endswith('.html'):
                thread = threading.Thread(target=process_file, args=(file_entry,))
                threads.append(thread)
                thread.start()

for thread in threads:
    thread.join()



/usr/local/lib/python3.10/dist-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


Sacamos los nombres de los html y les quitamos la extensión para poder buscar los indices más adelante. 

In [159]:
print(Name_Files)
print(len(Name_Files))
Name_Files_clean = []
for doc in Name_Files:
    Name_Files_clean.append(doc.replace('.html', ''))
print(Name_Files_clean)

['2010-83-007.html', '2010-83-028.html', '2010-83-027.html', '2010-83-006.html', '2010-83-001.html', '2010-83-024.html', '2010-83-017.html', '2010-83-071.html', '2010-83-077.html', '2010-83-050.html', '2010-83-076.html', '2010-83-085.html', '2010-83-092.html', '2010-83-089.html', '2010-83-069.html', '2010-83-056.html', '2010-83-052.html', '2010-83-029.html', '2010-83-030.html', '2010-83-083.html', '2010-69-058.html', '2010-69-063.html', '2010-69-080.html', '2010-69-019.html', '2010-69-084.html', '2010-69-023.html', '2010-69-077.html', '2010-69-006.html', '2010-69-000.html', '2010-69-054.html', '2010-69-010.html', '2010-69-064.html', '2010-69-059.html', '2010-69-003.html', '2010-69-018.html', '2010-69-044.html', '2010-69-050.html', '2010-79-031.html', '2010-79-095.html', '2010-79-001.html', '2010-79-027.html', '2010-79-051.html', '2010-79-086.html', '2010-79-071.html', '2010-79-048.html', '2010-79-054.html', '2010-79-050.html', '2010-79-082.html', '2010-79-049.html', '2010-79-037.html',

# Extracción de Nombres
Extraemos los nombres masculinos y femeninos para usarlos en El modulo de Reconocimiento de Entidades.

In [160]:

Nombres_total = []
for file in os.listdir(fn_nombres):
   Nombres = []
   d = os.path.join(fn_nombres, file)
   print(d)
   with open (d, 'r') as files :
    Nombres = files.read()
    Nombres = Nombres.splitlines()
    Nombres_total.extend(Nombres) #Añadimos los nombres a una lista

/content/drive/MyDrive/RAI/Practica_MOTOR/Motor_Recuperación_Evaluación/Person_names/female.txt
/content/drive/MyDrive/RAI/Practica_MOTOR/Motor_Recuperación_Evaluación/Person_names/male_v2.txt


In [161]:
print(Nombres_total)

['Abagael', 'Abagail', 'Abbe', 'Abbey', 'Abbi', 'Abbie', 'Abby', 'Abigael', 'Abigail', 'Abigale', 'Abra', 'Acacia', 'Ada', 'Adah', 'Adaline', 'Adara', 'Addie', 'Addis', 'Adel', 'Adela', 'Adelaide', 'Adele', 'Adelice', 'Adelina', 'Adelind', 'Adeline', 'Adella', 'Adelle', 'Adena', 'Adey', 'Adi', 'Adiana', 'Adina', 'Adora', 'Adore', 'Adoree', 'Adorne', 'Adrea', 'Adria', 'Adriaens', 'Adrian', 'Adriana', 'Adriane', 'Adrianna', 'Adrianne', 'Adrien', 'Adriena', 'Adrienne', 'Aeriel', 'Aeriela', 'Aeriell', 'Ag', 'Agace', 'Agata', 'Agatha', 'Agathe', 'Aggi', 'Aggie', 'Aggy', 'Agna', 'Agnella', 'Agnes', 'Agnese', 'Agnesse', 'Agneta', 'Agnola', 'Agretha', 'Aida', 'Aidan', 'Aigneis', 'Aila', 'Aile', 'Ailee', 'Aileen', 'Ailene', 'Ailey', 'Aili', 'Ailina', 'Ailyn', 'Aime', 'Aimee', 'Aimil', 'Aina', 'Aindrea', 'Ainslee', 'Ainsley', 'Ainslie', 'Ajay', 'Alaine', 'Alameda', 'Alana', 'Alanah', 'Alane', 'Alanna', 'Alayne', 'Alberta', 'Albertina', 'Albertine', 'Albina', 'Alecia', 'Aleda', 'Aleece', 'Aleecia

# Proceso de Stemming y Limpieza de carácteres en contenido de HTML

---

Recordar que el proceso del Stemming aumenta el recall que es una medida sobre el número de documentos que se pueden encontrar con una o varias consultas

 
Ademas interesa limpiar los conjuntos de palabras de cualquier carácter que no sirva para el cálculo de modelo vectorial. 

---




Función que realiza la tokenización de texto. 

In [162]:
def TOK(array):
  Lista=[]
  for a in array:
    obj =  WhitespaceTokenizer().tokenize(a.lower()) #tokenizamos por espacio y transformamos a minusculas. 
    Lista.append(obj)
  return Lista 

Función que realiza la eliminación de palabras vacias del texto

In [163]:
print(stopWords)
def StopWords(array):
    result=[]
    for a in array:
      for w in a:
        if w not in stopWords:
          result.append(w)
    return result

{'themselves', 'mustn', 'between', 'below', 'this', 'herself', 'has', 'if', 'because', 'as', 'during', 'how', "aren't", 'hers', 'own', 'won', 'to', 'above', 'my', 'your', 'what', "isn't", 'and', 'those', 'from', 'through', "you'd", 'with', "you've", 'most', 'by', 'yourselves', 'both', 've', 'a', 'here', 'down', 'too', 'very', 't', 'am', "didn't", "wasn't", "you'll", 'yours', 'their', 'will', 're', 'there', 'or', 'he', 'all', 'so', "won't", 'its', 'needn', 'each', "weren't", 'just', 'himself', 'ours', 'off', 'was', 'o', 'against', "mustn't", 'under', 'ourselves', 'again', 'up', 'why', 'than', "needn't", 'over', 'now', 'any', 'when', 'myself', 'couldn', 'is', 'itself', 'only', "should've", "couldn't", 'yourself', 'were', 'some', 'did', 'after', 'whom', 'him', 'd', 'aren', "wouldn't", 'who', 'nor', 'but', 'at', "mightn't", 'm', "haven't", "that'll", 'ma', 'are', 'i', 'theirs', 'that', "you're", "she's", 'for', 'isn', 'about', 'of', 'once', 'don', 'doing', 'not', 'y', 'haven', 'where', 'mi

Función que realiza un proceso de stemming y de limpieza de carácteres especiales

In [164]:
def stemList(array):
  Steem_list = []
  lista =""
  for a in array:
    a = snow_stemmer.stem(a)
    for i in special_characters:   #Utilizaremos la misma lista que carácteres que le pasamos al contenido de los htmls. 
      a=a.replace(i,"")
    Steem_list.append(a)
  return Steem_list

Con esta función limpiamos, tokenizamos y eliminamos las palabras vacias de los documentos htmls

In [165]:
New_Lista = []
#Definimos la lista de cáracteres espciales que queremos quitar (ver si hay que quitar números)
for l in Lista_Content:#Para cada lista dentro de el contenido (1 lista x cada html)
    aux = []
    lista = TOK(l)
    lista2 = StopWords(lista)
    for words in lista2:
          
            #stemmed_word = snow_stemmer.stem(words)# Hacemos stemming de esa palabra
            stemmed_word= ps.stem(words)
            #Quitar caracteres especiales
            for character in special_characters:
                stemmed_word = stemmed_word.replace(character, "")#En caso de coincidencía, se remplaza, si no hay, se ignora (método .replace())
            if stemmed_word != "":
              aux.append(stemmed_word) #Esa lista de añade a la nueva lista de contenido#.
    New_Lista.append(aux)

#Proceso de Stemming y Limpieza de carácteres de las consultas


---

En esta sección realizaremos el stemming de las tres consultas. Aqui necesitamos hacer el mismo proceso de stemming que en los htmls pero aplicandolo a las distintas consultas propuestas.


In [166]:
#Esta función se llama para ver si dentro de la consulta existe algún nombre presente en la lista de nombres. 
def MatchNombreConsuta(array,string):
  for a in array:
    if a in string:
      return True
  return False

In [167]:
lista_stem= []
Lista_Consultas_Entidades=[]
Entidad = False
ListEntity= ['who','Who','Which','which','whom','Whom']
#cada lista se añade a este array. Obtendremos un array de 3 consultas. 
#Para poder realizar el proceso de stemming se necesita una lista de palabras, siendo cada palabra una posición en la lista.
with open (fn_consultas, 'r') as fichero:
  d = BeautifulSoup(fichero.read(),"xml")
  consultas = d.find_all('title')
  for c in consultas: 
    consulta = c.text
    print(consulta)
    lista =[] #En esta lista iremos almacenando las palabras despues del proceso de stemming
    palabra_en_consulta = consulta.split()  # divide la lista en palabras
    
    if c not in Lista_Consultas_Entidades:
        Entidad = MatchNombreConsuta(ListEntity,consulta)
        Lista_Consultas_Entidades.append(Entidad)
    for p in palabra_en_consulta:
       if p not in stopWords:
          p =  ps.stem(p)
          #p = snow_stemmer.stem(p)
          for i in special_characters:   #Utilizaremos la misma lista que carácteres que le pasamos al contenido de los htmls. 
            p=p.replace(i,"")
          lista.append(p) # Si no añadimos el espacio entre cada palabra, obtenemos un conjunto de palabras juntas que no interesan
    lista_stem.append(lista)
   # añade las palabras a la lista de palabras. 

print(lista_stem)

print(Lista_Consultas_Entidades)

What attributes do the table and body tags have in XHTML?
How to connect a C# program with R?
What year was Facebook founded on?
Where are Google’s data-centers located?
How to change the HTTP headers in C#?
What free software is there for natural language processing?
Is the Open Document Format supported by Microsoft Wordpad?
How many web pages had Google indexed between 2008 and 2009?
What 2GB DDR3 memory modules can be bought for under 70€?
What laptop bags have airbag?
What is the default combination of Kensington cables?
What free software is available for Windows to convert mp3 files to wav?
What year was Windows XP released on?
Who won the first ACM Gerard Salton prize?
Who was Microsoft’s Chief Executive Officer in 2009?
What processor obtained the best score in 2009 for the Photoshop benchmark?
What graphics card obtained the best score in 2008 for the Battlefield 2142 benchmark?
What video game won Spike's best driving game award in 2006?
What laptops with AMD processor are o

# Cambio de nombre de las variables importantes

---

Tenemos las consultas limpias pasadas por el proceso de stemming y el contenido de cada html limpiado y stemmeado. 



In [168]:
#Cambiamos la terminología en base al tutorial dado, para una mejor comprensión y reutilización del codigo. 
#Cambiamos su formato para poder pasar las variables al modelo vectorial. 
corpus=[]
query=[]
Lista_Content2=[]
for f in Lista_Content:
  str = " ".join(f)
  Lista_Content2.append(str)

for f in New_Lista:
  str = " ".join(f)
  corpus.append(str)

for f in lista_stem:
  str = " ".join(f)
  query.append(str)

# Relevancía de los documentos HTML.

---

En esta sección recuperamos las distintas relevancías de los archivos htmls a partir de el archivo "union23.trel". 

Esta lista de relevancías llevan asociados el identificador de consulta/query, el nombre del documento html y la relevancía de dicho documento para dicha consulta. La relevancía se clasifica en trés categorías númericas que determinan el nivel de relevancía: 

-0: The document is not related to the topic. It may contain some common terms, but still not related to the topic

-1:The document is related to the topic, but does not satisfy the information need. It may contain a hyperlink to a relevant document.

-2:The document is related to the topic and does satisfy the information need.

In [169]:
Lista_Relevancia = []
#Nos interesa crear una lista de listas, donde cada lista es una fila del archivo .trel
with open (fn_relevancia, 'r') as fichero:
   for row in fichero: 
     f = row.split()
     Lista_Relevancia.append(f)
Lista_Relevancia.remove(Lista_Relevancia[0]) #La primera lista incluida en el conjunto de listas es el nombre de las columnas, que no nos interesan. 
#Obtenemos un lista de listas, cada lista tiene un ID del query asociado, el nombre del documento y la relevancía. 
print(Lista_Relevancia)
len(Lista_Relevancia)

[['1', '2010-53-005', '0'], ['1', '2010-45-017', '1'], ['1', '2010-35-079', '1'], ['1', '2010-63-007', '1'], ['1', '2010-26-047', '2'], ['1', '2010-33-078', '0'], ['1', '2010-89-058', '2'], ['1', '2010-51-008', '0'], ['1', '2010-56-028', '1'], ['1', '2010-50-086', '0'], ['1', '2010-00-049', '2'], ['1', '2010-36-033', '2'], ['1', '2010-24-084', '1'], ['1', '2010-09-042', '1'], ['1', '2010-44-031', '0'], ['1', '2010-18-051', '0'], ['1', '2010-70-047', '1'], ['1', '2010-55-033', '0'], ['1', '2010-00-001', '1'], ['1', '2010-64-085', '0'], ['1', '2010-02-026', '2'], ['1', '2010-33-098', '2'], ['1', '2010-34-089', '0'], ['1', '2010-58-061', '0'], ['1', '2010-53-076', '0'], ['1', '2010-68-026', '2'], ['1', '2010-03-017', '0'], ['1', '2010-19-049', '1'], ['1', '2010-46-094', '1'], ['1', '2010-41-051', '0'], ['1', '2010-34-109', '0'], ['1', '2010-19-042', '0'], ['1', '2010-36-073', '2'], ['1', '2010-71-017', '0'], ['1', '2010-42-074', '1'], ['1', '2010-05-037', '0'], ['1', '2010-39-003', '0'], 

2018

Nos interesa sacar todos los documentos que aparezcan en el union.trel que tengan asociados el topic que haya activado el modulo de reconocimiento de entidades. 

In [170]:
Lista_Relevancia_Topics_Entidades = []
#Lista_Consultas_Entidades
for a in Lista_Relevancia: 
   for i in range (0,len(Lista_Consultas_Entidades)):
    if Lista_Consultas_Entidades[i] == True and int(a[0]) == i+1:
      Lista_Relevancia_Topics_Entidades.append(a)
print(Lista_Relevancia_Topics_Entidades)
print(len(Lista_Relevancia_Topics_Entidades))

[['14', '2010-77-043', '0'], ['14', '2010-22-005', '2'], ['14', '2010-55-080', '1'], ['14', '2010-52-015', '1'], ['14', '2010-31-022', '1'], ['14', '2010-13-015', '0'], ['14', '2010-22-100', '0'], ['14', '2010-74-063', '1'], ['14', '2010-66-086', '0'], ['14', '2010-02-061', '1'], ['14', '2010-98-029', '0'], ['14', '2010-69-006', '1'], ['14', '2010-97-031', '1'], ['14', '2010-59-051', '1'], ['14', '2010-58-087', '2'], ['14', '2010-25-003', '0'], ['14', '2010-73-096', '1'], ['14', '2010-80-076', '0'], ['14', '2010-37-013', '0'], ['14', '2010-82-093', '0'], ['14', '2010-84-003', '2'], ['14', '2010-63-043', '1'], ['14', '2010-02-029', '1'], ['14', '2010-99-086', '2'], ['14', '2010-37-000', '0'], ['14', '2010-54-006', '0'], ['14', '2010-38-034', '0'], ['14', '2010-87-108', '1'], ['14', '2010-02-068', '1'], ['14', '2010-42-100', '1'], ['14', '2010-96-080', '0'], ['14', '2010-62-044', '0'], ['14', '2010-20-042', '0'], ['14', '2010-90-042', '1'], ['14', '2010-97-028', '0'], ['14', '2010-43-017

Sacamos los indices de los documentos para luego poderlos leer. 

In [171]:
Index_Name_File_Entidad = []
for a in Lista_Relevancia_Topics_Entidades:
  if a[1] in Name_Files_clean:
    Index_Name_File_Entidad.append(Name_Files_clean.index(a[1]))
print(Index_Name_File_Entidad)
print(len(Index_Name_File_Entidad))

[1833, 1732, 1332, 981, 549, 869, 1744, 1590, 366, 1812, 998, 27, 1932, 815, 275, 1080, 88, 1802, 1676, 533, 1885, 1179, 1820, 1621, 1674, 397, 1715, 1053, 1817, 1563, 1306, 852, 1567, 1493, 1941, 1602, 286, 769, 1404, 375, 166, 744, 493, 174, 999, 45, 651, 238, 1263, 857, 1722, 1264, 1301, 1129, 1059, 37, 368, 1410, 1837, 1746, 181, 903, 1695, 1356, 272, 1399, 1468, 717, 1767, 1254, 1622, 542, 1858, 722, 1071, 1750, 1629, 1472, 348, 566, 556, 363, 1805, 1091, 1702, 97, 875, 536, 867, 1385, 1723, 905, 1252, 197, 825, 656, 1017, 408, 728, 1450, 1061, 1491, 495, 35, 248, 941, 1169, 1731, 1422, 1775, 422, 293, 639, 1438, 977, 1255, 737, 479, 1656, 1369, 805, 1373, 555, 1742, 1128, 216, 1431, 68, 1043, 1871, 1694, 338, 623, 1612, 235, 1513, 1686, 1729, 1067, 1719, 559, 1272, 365, 1518, 714, 1383, 1577, 1699, 1718, 640, 1226, 604, 1645, 726, 529, 456, 834, 403, 1197, 686, 471, 1681, 419, 643, 1573, 890, 1379, 991, 4, 1603, 186, 524, 1572, 993, 1891, 1611, 252, 1755, 334, 1760, 1725, 939, 11

Con los indices concretos de los documentos que esten ligados al topic cuyo modulo de reconocimiento de entidadas haya activado, contamos las apariciones de entidades en esos documentos. 

Cabe destacar que usamos los nombres sin pasar por el proceso de stemming, asi como el conjunto de documentos sin limpiar ni tokenizar.

In [172]:
Docs_Coincidencia_Entidades = {}
for index in Index_Name_File_Entidad: 
  Docs_Coincidencia_Entidades[Name_Files_clean[index]] = 0  #Se inicializa cada documento en 0. 
  for name in Nombres_total:
    if name in Lista_Content2[index]:
     Docs_Coincidencia_Entidades[Name_Files_clean[index]] += 1

print(Docs_Coincidencia_Entidades) 

{'2010-77-043': 145, '2010-22-005': 74, '2010-55-080': 81, '2010-52-015': 63, '2010-31-022': 62, '2010-13-015': 45, '2010-22-100': 27, '2010-74-063': 119, '2010-66-086': 187, '2010-02-061': 13, '2010-98-029': 26, '2010-69-006': 0, '2010-97-031': 40, '2010-59-051': 104, '2010-58-087': 33, '2010-25-003': 68, '2010-73-096': 28, '2010-80-076': 72, '2010-37-013': 80, '2010-82-093': 41, '2010-84-003': 59, '2010-63-043': 99, '2010-02-029': 42, '2010-99-086': 450, '2010-37-000': 7, '2010-54-006': 46, '2010-38-034': 50, '2010-87-108': 125, '2010-02-068': 42, '2010-42-100': 15, '2010-96-080': 8, '2010-62-044': 49, '2010-20-042': 173, '2010-90-042': 34, '2010-97-028': 46, '2010-43-017': 15, '2010-09-017': 27, '2010-33-028': 94, '2010-44-082': 81, '2010-86-028': 81, '2010-11-083': 18, '2010-47-075': 7, '2010-64-101': 14, '2010-65-043': 99, '2010-98-060': 94, '2010-79-054': 5, '2010-85-078': 41, '2010-07-098': 4, '2010-32-009': 7, '2010-13-008': 26, '2010-38-072': 11, '2010-32-071': 34, '2010-96-05

#COSENO TF-IDF

---

Hemos elegido utilizar para los cálculos de métricas de efectividad únicamente 
el CosenoTFIDF. 

In [173]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english',
                                    min_df=1, #ignorar terminos que aparecen en muy pocos documentos
                                    )
matriz_tfidf = tfidf_vectorizer.fit_transform(corpus)
query_tfidf = tfidf_vectorizer.transform(query)

# Calcular productos escalares TFIDF
#####La RAM falla si intentamos calcular el producto escalar entre el conjunto de palabras de los documentos htmls y el conjunto de consultas.#########

cos_tfidf=cosine_similarity(query_tfidf, matriz_tfidf) 
#==>La matriz de cosenos es la consulta i x el documento j, luego cada fila son los cosenos de 1 consulta x 1965 documentos. hay un total de 15 consultas. 
print(cos_tfidf)
cos_tfidf.shape


[[0.00000000e+00 0.00000000e+00 4.09274520e-03 ... 5.26013716e-03
  3.23657627e-03 1.50709490e-03]
 [3.62574243e-03 1.61198470e-02 0.00000000e+00 ... 5.59156770e-03
  3.75621653e-03 9.01268635e-03]
 [3.14336901e-03 0.00000000e+00 5.07965712e-02 ... 1.29265356e-04
  8.46281213e-04 3.71349655e-02]
 ...
 [3.76500241e-01 4.04382913e-03 6.57254341e-03 ... 2.08163143e-03
  4.74006029e-03 1.85565232e-03]
 [4.09660133e-04 5.48287384e-02 4.27106647e-03 ... 3.03237555e-04
  3.56273429e-03 2.04598085e-03]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.39505525e-02
  9.87623584e-04 1.79802511e-04]]


(20, 1965)

# Union de estructuras

-Relacionar la matriz de cosenos con la lista de relevancía, para poder asignar a cada documento su respectivo coseno. 

==>La matriz de cosenos es la consulta i x el documento j, luego cada fila son los cosenos de 1 consulta x 1965 documentos. hay un total de 15 consultas. 

==>El array de relevancias tiene el nombre del documento junto con el id de la consulta y la relevancía a esta. 

==>Podemos sacar la posición de cada documento por el array de nombres de documentos html. (Names_Files)



Funcion que ordena un array de mayor a menor en función del valor en su tercera posición, en este caso el coseno. 

In [174]:
def sort(array): 
 return sorted(array,reverse=True, key=lambda x:x[3])

Función que combina el nombre de los documentos htmls con su correspondiente valor coseno en la matriz resultante de la vectorización

In [175]:
def matchNameCos(cos,array):
  i = 0
  Lista=[]
  for field in cos:
    lista=[]
    for c in field: 
      lista.append([array[i],c]) 
      i=i+1
    i=0
    Lista.append(lista)
  #Con esto quitamos el array anidado para poder trabajar mejor
  Lista= Lista[0]
  return Lista


In [176]:
Lista_nombres_cosenos=matchNameCos(cos_tfidf,Name_Files_clean)
print(Lista_nombres_cosenos)
print(len(Lista_nombres_cosenos))

[['2010-83-007', 0.0], ['2010-83-028', 0.0], ['2010-83-027', 0.004092745199316026], ['2010-83-006', 0.0], ['2010-83-001', 0.016290356644844], ['2010-83-024', 0.0029109473768430543], ['2010-83-017', 0.0], ['2010-83-071', 0.0], ['2010-83-077', 0.01031533977367998], ['2010-83-050', 0.042607513878558406], ['2010-83-076', 0.0], ['2010-83-085', 0.0], ['2010-83-092', 0.0], ['2010-83-089', 0.0], ['2010-83-069', 0.0], ['2010-83-056', 0.0], ['2010-83-052', 0.0], ['2010-83-029', 0.0], ['2010-83-030', 0.009440393729605796], ['2010-83-083', 0.0037483898240635825], ['2010-69-058', 0.0], ['2010-69-063', 0.0], ['2010-69-080', 0.00983993561829805], ['2010-69-019', 0.0], ['2010-69-084', 0.01032283864262207], ['2010-69-023', 0.002696808136293487], ['2010-69-077', 0.0], ['2010-69-006', 0.0017587465995907176], ['2010-69-000', 0.0], ['2010-69-054', 0.0], ['2010-69-010', 0.0], ['2010-69-064', 0.008825511158949742], ['2010-69-059', 0.0], ['2010-69-003', 0.0], ['2010-69-018', 0.0], ['2010-69-044', 0.0017273302

Creamos nuestra estructura final con la que vamos a trabajar al calcular las métricas de evaluación. Esta estructura se llama Final_List_Relevancia. Tambien creamos una extendida para luego poder premiar. 

In [177]:
#Ahora que tenemos los nombres y los cosenos, hay que relacionarlos con el array de relevancía. para crear una sola estructura. 

#Con esto creamos un diccionario donde el id de cada coseno es el nombre del documento. 
dict1 = {}
for i in Lista_nombres_cosenos:
    dict1[i[0]] = i[1]
print(dict1)

###################Esto de aqui abajo igual lo podemos convertir a función ==> 

# Función para obtener la clave de ordenamiento
def ordenar_por_valores(item):
    return (item[0], -item[3])  # Tupla con los dos criterios de ordenamiento

Final_List_Relevancia=[]
Final_List_Relevancia_Extended = [] #La lista que en el siguiente paso almacenara los nuevos cosenos. 
for i in Lista_Relevancia:
  if i[1] in dict1:
    # Estructura: de la lista:"id_topic", "Nombre doc", "Relevancía", "valor coseno"
    Final_List_Relevancia.append([i[0], i[1], i[2], dict1[i[1]]])
    Final_List_Relevancia_Extended.append([i[0], i[1], i[2], dict1[i[1]]])

Final_List_Relevancia_ordenada = sorted(Final_List_Relevancia,key=ordenar_por_valores)
# Ordenar la lista por el primer y último valor del parámetro
#Final_List_Relevancia_ordOrdenarenada = sorted(Final_List_Relevancia,key=ordenar_por_valores)

print(Final_List_Relevancia)
print(len(Final_List_Relevancia))


{'2010-83-007': 0.0, '2010-83-028': 0.0, '2010-83-027': 0.004092745199316026, '2010-83-006': 0.0, '2010-83-001': 0.016290356644844, '2010-83-024': 0.0029109473768430543, '2010-83-017': 0.0, '2010-83-071': 0.0, '2010-83-077': 0.01031533977367998, '2010-83-050': 0.042607513878558406, '2010-83-076': 0.0, '2010-83-085': 0.0, '2010-83-092': 0.0, '2010-83-089': 0.0, '2010-83-069': 0.0, '2010-83-056': 0.0, '2010-83-052': 0.0, '2010-83-029': 0.0, '2010-83-030': 0.009440393729605796, '2010-83-083': 0.0037483898240635825, '2010-69-058': 0.0, '2010-69-063': 0.0, '2010-69-080': 0.00983993561829805, '2010-69-019': 0.0, '2010-69-084': 0.01032283864262207, '2010-69-023': 0.002696808136293487, '2010-69-077': 0.0, '2010-69-006': 0.0017587465995907176, '2010-69-000': 0.0, '2010-69-054': 0.0, '2010-69-010': 0.0, '2010-69-064': 0.008825511158949742, '2010-69-059': 0.0, '2010-69-003': 0.0, '2010-69-018': 0.0, '2010-69-044': 0.001727330256198879, '2010-69-050': 0.0, '2010-79-031': 0.0, '2010-79-095': 0.0, '

In [178]:
#Sistema de premios por número de entidades encontradas en el documento. 
def premiarCos(num,cos):
  if num > 0 and num < 30:
    return cos + 0.1
  if num > 30 and num < 60:
    return cos + 0.2
  if num > 60 and num < 90:
    return cos + 0.25
  if num > 90 and num < 120 :
    return cos + 0.3
  if num > 120 and num < 150:
    return cos + 0.35
  if num > 150 and num < 200:
    return cos + 0.4
  if num > 200 and num < 300:
    return cos + 0.5
  if num > 300and num < 500:
    return cos + 0.6
  if num > 500:
    return cos + 0.7
  else: 
    return cos

In [179]:
#Premiamos los documentos que tengan asociados el topic que haya activado el modulo. 
for a in Final_List_Relevancia_Extended:
  if a[1] in Docs_Coincidencia_Entidades:
     a[3] = premiarCos(Docs_Coincidencia_Entidades[a[1]],a[3])

print(Final_List_Relevancia)
print(Final_List_Relevancia_Extended)


[['1', '2010-53-005', '0', 0.02398571415483739], ['1', '2010-45-017', '1', 0.004649488398161951], ['1', '2010-35-079', '1', 0.0], ['1', '2010-63-007', '1', 0.009166342945336565], ['1', '2010-26-047', '2', 0.0], ['1', '2010-33-078', '0', 0.0], ['1', '2010-89-058', '2', 0.18046538478115695], ['1', '2010-51-008', '0', 0.00011844454471131949], ['1', '2010-56-028', '1', 0.005176116906903665], ['1', '2010-50-086', '0', 0.004021192261300123], ['1', '2010-00-049', '2', 0.01193149319536173], ['1', '2010-36-033', '2', 0.39649558919347655], ['1', '2010-24-084', '1', 0.003196203828621753], ['1', '2010-09-042', '1', 0.04424024891881874], ['1', '2010-44-031', '0', 0.022682294926671915], ['1', '2010-18-051', '0', 0.0], ['1', '2010-70-047', '1', 0.002606490006684234], ['1', '2010-55-033', '0', 0.0066066074132219745], ['1', '2010-00-001', '1', 0.0], ['1', '2010-64-085', '0', 0.0], ['1', '2010-02-026', '2', 0.007228213651984634], ['1', '2010-33-098', '2', 0.028146105811852257], ['1', '2010-34-089', '0',

In [180]:
#Esta función devuelve una lista de documentos en función del indice que le indiques.
#EJ: indice 3 ==> documentos de la consulta 3 sin ordenar. 
def lista_topics (indice,array):
  Lista_top=[]
  for elem in array:
    if int(elem[0]) == indice:
      #print(elem[2])
      Lista_top.append(elem)  
  return Lista_top


In [181]:
#extrae num valores de un array determinado.
def Extract_values(num,array):
  Lista_valores_extraidos=[]
  array = sort(array)
  for i in range(0,num):
      Lista_valores_extraidos.append(array[i])
  return Lista_valores_extraidos
  

In [182]:
#Se le pasa la lista recuperada con la función lista_topics y la lista de todos los documentos ==> Final_List_Relevancía
#Estructura: de la lista: "Nombre doc","valor coseno","id_topic","Relevancía"
def Comprobar_num_relevantes(array):
  counter = 0
  for elem in array:
      if int(elem[2]) == 1 or int(elem[2]) == 2:
        counter += 1 
  return counter

#Precisión en cortes 5, 10 (relevancia mínima 1).




In [183]:
from scipy.optimize._lsq.common import reflective_transformation
# Calculo de la precison en cortes 5 y 10
Suma_precisión_5=0
Suma_precisión_10=0
prec_5 = []
prec_10 = []
relevantes_en_5 = [] 
relevantes_en_10 = [] 
#Variables para la precisión extendida al modulo. 
Suma_precisión_5_E=0
Suma_precisión_10_E=0
prec_5_E= []
prec_10_E = []
relevantes_en_5_E = [] 
relevantes_en_10_E = [] 

#Función para cada topic, crear una colección por id los resultados igual. 
for item in range(1,len(cos_tfidf)+1):
  relevantes_en_5.append(Comprobar_num_relevantes(Extract_values(5,lista_topics(item,Final_List_Relevancia)))) #docs relevantes en punto 5 de cada topic
  relevantes_en_5_E.append(Comprobar_num_relevantes(Extract_values(5,lista_topics(item,Final_List_Relevancia_Extended))))
  relevantes_en_10.append(Comprobar_num_relevantes(Extract_values(10,lista_topics(item,Final_List_Relevancia)))) #docs relevantes en punto 10 de cada topic
  relevantes_en_10_E.append(Comprobar_num_relevantes(Extract_values(10,lista_topics(item,Final_List_Relevancia_Extended))))


w=0
for item in range(1,len(cos_tfidf)+1):
  prec_5.append((relevantes_en_5[w])/5)
  prec_5_E.append((relevantes_en_5_E[w])/5)
  prec_10.append((relevantes_en_10[w])/10)
  prec_10_E.append((relevantes_en_10_E[w])/10)

  w=w+1

#Función para la media. 
for i in range (1,len(cos_tfidf)+1):
  Suma_precisión_5 += Comprobar_num_relevantes(Extract_values(5,lista_topics(i,Final_List_Relevancia)))/5
  Suma_precisión_10 += Comprobar_num_relevantes(Extract_values(10,lista_topics(i,Final_List_Relevancia)))/10
  Suma_precisión_5_E += Comprobar_num_relevantes(Extract_values(5,lista_topics(i,Final_List_Relevancia_Extended)))/5
  Suma_precisión_10_E += Comprobar_num_relevantes(Extract_values(10,lista_topics(i,Final_List_Relevancia_Extended)))/10

precision_5_media= Suma_precisión_5/(len (cos_tfidf))
precision_10_media= Suma_precisión_10/(len (cos_tfidf))
precision_5_media_E= Suma_precisión_5_E/(len (cos_tfidf))
precision_10_media_E= Suma_precisión_10_E/(len (cos_tfidf))

print("Media precision en corte 5: ", '{:.4f}'.format(precision_5_media))
print("Media precision en corte 10: ", '{:.4f}'.format(precision_10_media))
print()
print("Media precision en corte 5 con Modulo activo: ", '{:.4f}'.format(precision_5_media_E))
print("Media precision en corte 10 con Modulo activo: ", '{:.4f}'.format(precision_10_media_E))

print("LA PRECISIÓN PARA CADA TOPIC EN EL PUNTO DE CORTE 5 ES, POR ORDEN DE TOPIC:")
z=1
for item in prec_5:
  print("Topic", z,":", "{:.4f}".format(item))
  z=z+1

print()

print("LA PRECISIÓN PARA CADA TOPIC EN EL PUNTO DE CORTE 10 ES, POR ORDEN DE TOPIC:")
z=1
for item in prec_10:
  print("Topic", z,":", "{:.4f}".format(item))
  z=z+1


Media precision en corte 5:  0.5000
Media precision en corte 10:  0.4800

Media precision en corte 5 con Modulo activo:  0.5200
Media precision en corte 10 con Modulo activo:  0.4750
LA PRECISIÓN PARA CADA TOPIC EN EL PUNTO DE CORTE 5 ES, POR ORDEN DE TOPIC:
Topic 1 : 0.6000
Topic 2 : 0.4000
Topic 3 : 0.4000
Topic 4 : 0.4000
Topic 5 : 0.4000
Topic 6 : 0.2000
Topic 7 : 0.6000
Topic 8 : 0.8000
Topic 9 : 0.6000
Topic 10 : 0.4000
Topic 11 : 0.0000
Topic 12 : 1.0000
Topic 13 : 1.0000
Topic 14 : 0.4000
Topic 15 : 0.4000
Topic 16 : 0.8000
Topic 17 : 0.6000
Topic 18 : 0.4000
Topic 19 : 0.4000
Topic 20 : 0.2000

LA PRECISIÓN PARA CADA TOPIC EN EL PUNTO DE CORTE 10 ES, POR ORDEN DE TOPIC:
Topic 1 : 0.6000
Topic 2 : 0.2000
Topic 3 : 0.5000
Topic 4 : 0.5000
Topic 5 : 0.4000
Topic 6 : 0.3000
Topic 7 : 0.6000
Topic 8 : 0.6000
Topic 9 : 0.7000
Topic 10 : 0.3000
Topic 11 : 0.0000
Topic 12 : 0.8000
Topic 13 : 0.8000
Topic 14 : 0.5000
Topic 15 : 0.3000
Topic 16 : 0.6000
Topic 17 : 0.5000
Topic 18 : 0.70

#  Exhaustividad en cortes 5, 10 (relevancia mínima 1).

In [184]:
Suma_exhaustidivad_5=0
Suma_exhaustidivad_10=0
Suma_exhaustidivad_5_E=0
Suma_exhaustidivad_10_E=0

recall_en_5 = []
recall_en_10 = []
recall_en_5_E = []
recall_en_10_E = []
#Función para cada topic crear una colección por id los resultados igual.
w=0
for item in range(1,len(cos_tfidf)+1):
  recall_en_5.append((relevantes_en_5[w])/Comprobar_num_relevantes(lista_topics(i,Lista_Relevancia)))
  recall_en_5_E.append((relevantes_en_5[w])/Comprobar_num_relevantes(lista_topics(i,Lista_Relevancia)))
  recall_en_10.append((relevantes_en_10[w])/Comprobar_num_relevantes(lista_topics(i,Lista_Relevancia)))
  recall_en_10_E.append((relevantes_en_10[w])/Comprobar_num_relevantes(lista_topics(i,Lista_Relevancia)))
  w=w+1


#**implementar**

#Función para la media
for i in range (1,len(cos_tfidf)+1):
  Suma_exhaustidivad_5 += Comprobar_num_relevantes(Extract_values(5,lista_topics(i,Final_List_Relevancia)))/Comprobar_num_relevantes(lista_topics(i,Lista_Relevancia))
  Suma_exhaustidivad_10 += Comprobar_num_relevantes(Extract_values(10,lista_topics(i,Final_List_Relevancia)))/Comprobar_num_relevantes(lista_topics(i,Lista_Relevancia))
  Suma_exhaustidivad_5_E += Comprobar_num_relevantes(Extract_values(5,lista_topics(i,Final_List_Relevancia_Extended)))/Comprobar_num_relevantes(lista_topics(i,Lista_Relevancia))
  Suma_exhaustidivad_10_E += Comprobar_num_relevantes(Extract_values(10,lista_topics(i,Final_List_Relevancia_Extended)))/Comprobar_num_relevantes(lista_topics(i,Lista_Relevancia))

exhaustidivad_5_media=  Suma_exhaustidivad_5/(len(cos_tfidf))
exhaustidivad_10_media= Suma_exhaustidivad_10/(len(cos_tfidf))
exhaustidivad_5_media_E=  Suma_exhaustidivad_5_E/(len(cos_tfidf))
exhaustidivad_10_media_E= Suma_exhaustidivad_10_E/(len(cos_tfidf))

print("Media exhaustidivad en corte 5: ", '{:.4f}'.format(exhaustidivad_5_media))
print("Media exhaustidivad en corte 10: ", '{:.4f}'.format(exhaustidivad_10_media))
print("Media exhaustidivad en corte 5 EXTENDIDO: ", '{:.4f}'.format(exhaustidivad_5_media_E))
print("Media exhaustidivad en corte 10 EXTENDIDO: ", '{:.4f}'.format(exhaustidivad_10_media_E))

print("LA EXHAUSTIVIDAD PARA CADA TOPIC EN EL PUNTO DE CORTE 5 ES, POR ORDEN DE TOPIC:")
z=1
for item in recall_en_5:
  print("Topic", z,":", "{:.4f}".format(item))
  z=z+1

print()

print("LA EXHAUSTIVIDAD PARA CADA TOPIC EN EL PUNTO DE CORTE 10 ES, POR ORDEN DE TOPIC:")
z=1
for item in recall_en_10:
  print("Topic", z,":", "{:.4f}".format(item))
  z=z+1

Media exhaustidivad en corte 5:  0.0524
Media exhaustidivad en corte 10:  0.0945
Media exhaustidivad en corte 5 EXTENDIDO:  0.0550
Media exhaustidivad en corte 10 EXTENDIDO:  0.0938
LA EXHAUSTIVIDAD PARA CADA TOPIC EN EL PUNTO DE CORTE 5 ES, POR ORDEN DE TOPIC:
Topic 1 : 0.1111
Topic 2 : 0.0741
Topic 3 : 0.0741
Topic 4 : 0.0741
Topic 5 : 0.0741
Topic 6 : 0.0370
Topic 7 : 0.1111
Topic 8 : 0.1481
Topic 9 : 0.1111
Topic 10 : 0.0741
Topic 11 : 0.0000
Topic 12 : 0.1852
Topic 13 : 0.1852
Topic 14 : 0.0741
Topic 15 : 0.0741
Topic 16 : 0.1481
Topic 17 : 0.1111
Topic 18 : 0.0741
Topic 19 : 0.0741
Topic 20 : 0.0370

LA EXHAUSTIVIDAD PARA CADA TOPIC EN EL PUNTO DE CORTE 10 ES, POR ORDEN DE TOPIC:
Topic 1 : 0.2222
Topic 2 : 0.0741
Topic 3 : 0.1852
Topic 4 : 0.1852
Topic 5 : 0.1481
Topic 6 : 0.1111
Topic 7 : 0.2222
Topic 8 : 0.2222
Topic 9 : 0.2593
Topic 10 : 0.1111
Topic 11 : 0.0000
Topic 12 : 0.2963
Topic 13 : 0.2963
Topic 14 : 0.1852
Topic 15 : 0.1111
Topic 16 : 0.2222
Topic 17 : 0.1852
Topic 18

# Valor-F basado en los puntos de corte anteriores, 5 y 10.

In [185]:
Fv_5 = []
Fv_10 = []
Fv_5_E = []
Fv_10_E = []
##########################Corte 5###############################
x=0
for item in range(1,len(cos_tfidf)+1):
  if ((prec_5[x]+recall_en_5[x]))==0:
    Fv_5 .append(0)
    x=x+1
  else:
    Fv_5 .append(2*(((prec_5[x])*(recall_en_5[x]))/((prec_5[x])+(recall_en_5[x]))))
    x=x+1

##########################Corte 5 Extendido###############################
#for item in range(1,len(cos_tfidf)+1):
 # if ((prec_5_E[x]+recall_en_5_E[x]))==0:
  #  Fv_5_E.append(0)
   # x=x+1
  #else:
   # Fv_5_E.append(2*(((prec_5_E[x])*(recall_en_5_E[x]))/((prec_5_E[x])+(recall_en_5_E[x]))))
    #x=x+1
##########################Corte 10 Extendido###############################
x=0
for item in range(1,len(cos_tfidf)+1):
  if ((prec_10_E[x]+recall_en_10_E[x]))==0:
    Fv_10_E.append(0)
    x=x+1
  else:
    Fv_10_E.append(2*(((prec_10_E[x])*(recall_en_10_E[x]))/((prec_10_E[x])+(recall_en_10_E[x]))))
    x=x+1

print("EL VALOR-F PARA CADA TOPIC EN EL PUNTO DE CORTE 5 ES, POR ORDEN DE TOPIC:")
r=1
for item in Fv_5:
  print("Topic", r,":", "{:.4f}".format(item))
  r=r+1
print()

##########################Corte 10###############################
x=0
for item in range(1,len(cos_tfidf)+1):
  if ((prec_10[x]+recall_en_10[x]))==0:
    Fv_10.append(0)
    x=x+1
  else:
    Fv_10.append(2*(((prec_10[x])*(recall_en_10[x]))/((prec_10[x])+(recall_en_10[x]))))
    x=x+1

print("EL VALOR-F PARA CADA TOPIC EN EL PUNTO DE CORTE 10 ES, POR ORDEN DE TOPIC:")
r=1
for item in Fv_10:
  print("Topic", r,":", "{:.4f}".format(item))
  r=r+1
print()


ValorF_5 = ((2*precision_5_media * exhaustidivad_5_media)/(precision_5_media + exhaustidivad_5_media))
ValorF_10= ((2*precision_10_media * exhaustidivad_10_media)/(precision_10_media + exhaustidivad_10_media))
ValorF_5_E = ((2*precision_5_media_E * exhaustidivad_5_media_E)/(precision_5_media_E + exhaustidivad_5_media_E))
ValorF_10_E= ((2*precision_10_media_E * exhaustidivad_10_media_E)/(precision_10_media_E + exhaustidivad_10_media_E))

print('Valor F corte 5: '  , '{:.4f}'.format(ValorF_5))
print('Valor F corte 10: ', '{:.4f}'.format(ValorF_10))
print('Valor F Extendido corte 5: '  , '{:.4f}'.format(ValorF_5_E))
print('Valor F Extendido corte 10: ', '{:.4f}'.format(ValorF_10_E))




EL VALOR-F PARA CADA TOPIC EN EL PUNTO DE CORTE 5 ES, POR ORDEN DE TOPIC:
Topic 1 : 0.1875
Topic 2 : 0.1250
Topic 3 : 0.1250
Topic 4 : 0.1250
Topic 5 : 0.1250
Topic 6 : 0.0625
Topic 7 : 0.1875
Topic 8 : 0.2500
Topic 9 : 0.1875
Topic 10 : 0.1250
Topic 11 : 0.0000
Topic 12 : 0.3125
Topic 13 : 0.3125
Topic 14 : 0.1250
Topic 15 : 0.1250
Topic 16 : 0.2500
Topic 17 : 0.1875
Topic 18 : 0.1250
Topic 19 : 0.1250
Topic 20 : 0.0625

EL VALOR-F PARA CADA TOPIC EN EL PUNTO DE CORTE 10 ES, POR ORDEN DE TOPIC:
Topic 1 : 0.3243
Topic 2 : 0.1081
Topic 3 : 0.2703
Topic 4 : 0.2703
Topic 5 : 0.2162
Topic 6 : 0.1622
Topic 7 : 0.3243
Topic 8 : 0.3243
Topic 9 : 0.3784
Topic 10 : 0.1622
Topic 11 : 0.0000
Topic 12 : 0.4324
Topic 13 : 0.4324
Topic 14 : 0.2703
Topic 15 : 0.1622
Topic 16 : 0.3243
Topic 17 : 0.2703
Topic 18 : 0.3784
Topic 19 : 0.2703
Topic 20 : 0.1081

Valor F corte 5:  0.0949
Valor F corte 10:  0.1580
Valor F Extendido corte 5:  0.0996
Valor F Extendido corte 10:  0.1567


Reciprocal Rank Rel 1

In [186]:
#Extraer 50-100 documentos para cada topic ordenados de mayor a menor coseno, el rango es equivalente al primer documento relevante de esos 100. 
 #Es decir hay 15 rankins distintos. Habria que usar la lista de Lista_nombres_cosenos pero para no complicarnos vamos a usar la Final_List_Relevancia
 #Esos documentos extraidos, verificar que se encuentran en el union.trel es decir en Lista_Relevancia o no, si no estan no son relevantes y se pasa a la siguiente posición.
 #Si estan, tienen que tener relevancía mayor o igual a la indicada ( 1, 2) ==>  relevant_doc por ej
 #que como itera del 1 al x la primera coincidencía que encuentre devuelve su posición en el array. 

def reciprocal_rank_posicion(array,relevancia):
  posición = 1
  Found = False
  for elem in array:
    if int(elem[2])>= relevancia and Found == False: 
          Found = True
    elif  int(elem[2]) < relevancia and Found == False: 
          posición +=1
          
  return posición
  
 

In [187]:
recc_1 = 0
recc_2 = 0

for i in range (1,len(cos_tfidf)+1):
    recc_1 += 1/reciprocal_rank_posicion(Extract_values(100,lista_topics(i,Final_List_Relevancia)),1)
    recc_2 += 1/reciprocal_rank_posicion(Extract_values(100,lista_topics(i,Final_List_Relevancia)),2)

recc_1 = recc_1/len (cos_tfidf)
recc_2 = recc_2/len (cos_tfidf)

print('Reciprocal rank 1 media: '  , '{:.4f}'.format(recc_1))
print('Reciprocal rank 2 media: ', '{:.4f}'.format(recc_2))

Reciprocal rank 1 media:  0.7327
Reciprocal rank 2 media:  0.3922


In [188]:
############## RANGO RECÍPROCO (min 1) ###################

# Guardamos la posición del primer documento con relevancia 1 o 2 de los 15 topics
def extract_first_docs_from_topics(data):
    topics = set([item[0] for item in data])
    docs = []
    for topic in topics:
        prev_topic = None
        pos = 1
        for i, item in enumerate(data):
            
            if topic != item[0]:
                continue
            if prev_topic != item[0] and (item[2] == '1' or item[2] == '2'):
                docs.append(pos)
                prev_topic = item[0]
                break
            pos += 1
    return docs


posiciones = extract_first_docs_from_topics(Final_List_Relevancia)
print("Posiciones de los primeros archivos relevantes de cada topic:", posiciones)


reciprocal_rank_min_1=[]
for i in range(0,len(cos_tfidf)):
  reciprocal_rank_min_1.append(1/posiciones[i])
print("EL RANGO RECÍPROCO PARA RELEVANCIA MÍNIMA 1 PARA CADA TOPIC ES:")
o=1
for item in reciprocal_rank_min_1:
  print("Topic", o,":", "{:.4f}".format(item))
  o=o+1
print()

posiciones_E = extract_first_docs_from_topics(Final_List_Relevancia_Extended)
print("Posiciones de los primeros archivos relevantes de cada topic:", posiciones)
#Recorre todos los temas en los datos y para cada tema, encuentra el índice del primer documento relevante para ese tema. 
#Luego, si se encuentra al menos un documento relevante, calcula el rango recíproco de relevancia 1 y lo agrega a una lista de rangos. 
#Finalmente, se devuelve el promedio de todos los rangos.
suma_rec_1 = 0
for i in range(0,len(cos_tfidf)):
  suma_rec_1 = suma_rec_1 + (1/posiciones[i])
media_Rec_1 = suma_rec_1/len(cos_tfidf)

suma_rec_e = 0
for i in range(0,len(cos_tfidf)):
  suma_rec_e = suma_rec_e + (1/posiciones_E[i])
media_Rec_E = suma_rec_e/len(cos_tfidf)

print("La media del Rango Recuiproco de relevancia minima 1 es: ", media_Rec_1)
print("La media del Rango Recuiproco de relevancia minima 1 Extendido es: ", media_Rec_E)

Posiciones de los primeros archivos relevantes de cada topic: [1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 5, 2, 1, 2, 1, 1, 1, 1, 2, 4]
EL RANGO RECÍPROCO PARA RELEVANCIA MÍNIMA 1 PARA CADA TOPIC ES:
Topic 1 : 1.0000
Topic 2 : 1.0000
Topic 3 : 1.0000
Topic 4 : 0.5000
Topic 5 : 0.5000
Topic 6 : 0.5000
Topic 7 : 0.5000
Topic 8 : 0.5000
Topic 9 : 0.5000
Topic 10 : 1.0000
Topic 11 : 0.2000
Topic 12 : 0.5000
Topic 13 : 1.0000
Topic 14 : 0.5000
Topic 15 : 1.0000
Topic 16 : 1.0000
Topic 17 : 1.0000
Topic 18 : 1.0000
Topic 19 : 0.5000
Topic 20 : 0.2500

Posiciones de los primeros archivos relevantes de cada topic: [1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 5, 2, 1, 2, 1, 1, 1, 1, 2, 4]
La media del Rango Recuiproco de relevancia minima 1 es:  0.6975
La media del Rango Recuiproco de relevancia minima 1 Extendido es:  0.6975


Reciprocal Rank Rel 2 

In [189]:
############## RANGO RECÍPROCO (min 2) ###################

# Guardamos la posición del primer documento con relevancia 2 de los 15 topics
def extract_first_docs_from_topics(data):
    topics = set([item[0] for item in data])
    docs = []
    for topic in topics:
        prev_topic = None
        pos = 1
        for i, item in enumerate(data):
            if topic != item[0]:
                continue
            if  prev_topic != item[0] and item[2] == '2':
                docs.append(pos)
                prev_topic = item[0]
                break
            pos += 1
    return docs

posiciones = extract_first_docs_from_topics(Final_List_Relevancia)
print("Posiciones de los primeros archivos relevantes de cada topic:", posiciones)

reciprocal_rank_min_2=[]
for i in range(0,len(cos_tfidf)):
  reciprocal_rank_min_2.append(1/posiciones[i])
print("EL RANGO RECÍPROCO PARA RELEVANCIA MÍNIMA 2 PARA CADA TOPIC ES:")
o=1
for item in reciprocal_rank_min_2:
  print("Topic", o,":", "{:.4f}".format(item))
  o=o+1
print()

posiciones_E = extract_first_docs_from_topics(Final_List_Relevancia_Extended)
print("Posiciones de los primeros archivos relevantes de cada topic:", posiciones)
#Recorre todos los temas en los datos y para cada tema, encuentra el índice del primer documento relevante para ese tema. 
#Luego, si se encuentra al menos un documento relevante, calcula el rango recíproco de relevancia 1 y lo agrega a una lista de rangos. 
#Finalmente, se devuelve el promedio de todos los rangos.
suma_rec_2 = 0
for i in range(0,len(cos_tfidf)):
  suma_rec_2 = suma_rec_2 + (1/posiciones[i])
media_Rec_2 = suma_rec_2/len(cos_tfidf)

suma_rec_2_e = 0
for i in range(0,len(cos_tfidf)):
  suma_rec_2_e = suma_rec_2_e + (1/posiciones_E[i])
media_Rec_2_E = suma_rec_2_e/len(cos_tfidf)
print("La media del Rango Reciproco de relevancia minima 2 Extendido es: ", media_Rec_2_E)

Posiciones de los primeros archivos relevantes de cada topic: [3, 1, 1, 5, 2, 2, 2, 2, 8, 44, 10, 21, 1, 4, 17, 1, 12, 8, 2, 6]
EL RANGO RECÍPROCO PARA RELEVANCIA MÍNIMA 2 PARA CADA TOPIC ES:
Topic 1 : 0.3333
Topic 2 : 1.0000
Topic 3 : 1.0000
Topic 4 : 0.2000
Topic 5 : 0.5000
Topic 6 : 0.5000
Topic 7 : 0.5000
Topic 8 : 0.5000
Topic 9 : 0.1250
Topic 10 : 0.0227
Topic 11 : 0.1000
Topic 12 : 0.0476
Topic 13 : 1.0000
Topic 14 : 0.2500
Topic 15 : 0.0588
Topic 16 : 1.0000
Topic 17 : 0.0833
Topic 18 : 0.1250
Topic 19 : 0.5000
Topic 20 : 0.1667

Posiciones de los primeros archivos relevantes de cada topic: [3, 1, 1, 5, 2, 2, 2, 2, 8, 44, 10, 21, 1, 4, 17, 1, 12, 8, 2, 6]
La media del Rango Reciproco de relevancia minima 2 Extendido es:  0.4006251591545708


Average Precision@100

In [190]:
def get_relevant_positions(docs): #con esta funcion vamos a guardar en 15 filas distintas las posiciones de aquellos documentos recuperados por cada topic y lo guardaremos abajo en la lista posiciones_relevantes
    positions = [[] for _ in range(20)]
    current_topic = 1
    current_position = 1
    for doc in docs:
        topic = int(doc[0])
        relevance = int(doc[2])
        if topic != current_topic:
            current_topic = topic
            current_position = 1
        if relevance in [1, 2]:
            positions[current_topic - 1].append(current_position)
        current_position += 1
    return positions

posiciones_relevantes = get_relevant_positions(Final_List_Relevancia_ordenada)
posiciones_relevantes_E = get_relevant_positions(Final_List_Relevancia_Extended)

AP_100 = []
for i in range(0,len(cos_tfidf)):
    suma = 0
    contador = 1
    for j in range(len(posiciones_relevantes[i])):
        suma += contador/posiciones_relevantes[i][j]
        contador += 1
    AP_100.append(suma/len(posiciones_relevantes[i]))


AP_100_E = []
for i in range(0,len(cos_tfidf)):
    suma = 0
    contador = 1
    for j in range(len(posiciones_relevantes_E[i])):
        suma += contador/posiciones_relevantes_E[i][j]
        contador += 1
    AP_100_E.append(suma/len(posiciones_relevantes_E[i]))
# Imprimir el valor de Average Precision@100 para cada consulta

print()
for i, ap in enumerate(AP_100):
    print("Topic {}: {:.4f}".format(i+1, ap))
    
# Imprimir el valor promedio de Average Precision@100 para todas las consultas
print(f"Average Precision@100 promedio: {sum(AP_100) / len(AP_100):.4f}")

# Imprimir el valor promedio de Average Precision@100 para todas las consultas
print(f"Average Precision@100 promedio Extendido: {sum(AP_100_E) / len(AP_100_E):.4f}")


Topic 1: 0.5088
Topic 2: 0.2015
Topic 3: 0.5262
Topic 4: 0.4064
Topic 5: 0.3845
Topic 6: 0.3765
Topic 7: 0.6164
Topic 8: 0.5864
Topic 9: 0.6054
Topic 10: 0.4571
Topic 11: 0.2628
Topic 12: 0.8813
Topic 13: 0.8043
Topic 14: 0.6328
Topic 15: 0.5206
Topic 16: 0.7035
Topic 17: 0.4780
Topic 18: 0.7721
Topic 19: 0.5993
Topic 20: 0.2929
Average Precision@100 promedio: 0.5308
Average Precision@100 promedio Extendido: 0.5431


nDCG en cortes 10, 100 en base 2

In [191]:
def discounted_acumulated_gain(array):
  
  sumatorio=0                                 # lo utilizaremos para almacenar los nombres de los documentos recuperados, luego buscaremos estos nombres en el union.trel                                                      # ademas los añadimos a la lista de documentos recuperados
  for elemento in array:                                   # recorremos la lista que contiene la info del union.trel
    if(int(elemento[2])>=1):          # Si el elemento leido esta entre los elementos recuperados, y tiene una relevancia de uno o mas aumentamos el contador de los documentos relevantes   
      if (array.index(elemento)==0):                  # si el documento leido es el primero que se recupera no realizo el cociente del, tal como indica la formula
        sumatorio+=((int(elemento[2])))
      else:
        sumatorio+=((int(elemento[2]))/(math.log((array.index(elemento)+1),2)))  # Se calcula el cociente entre la relevancia del documentos y el logaritmo de la posicion donde se encuentra
  return sumatorio


In [192]:
def ideal_discounted_acumulated_gain(numero_de_la_consulta, corte):     # numero de la consulta es el numero de la query que estamos buscando
  sumatorio=0
  lista_documentos_ya_evaluados_2=[]                                          
  lista_documentos_ya_evaluados_1=[]    
  
  for elemento in Lista_Relevancia:                                           # leemos en el trel todos los documentos para esa consulta, separandolos en dos lista una para la rel 1 y otro para el rel 2
    if (int(elemento[0])==numero_de_la_consulta):                   
        if (int(elemento [2])==2):            
          lista_documentos_ya_evaluados_2.append(elemento)   
        if (int(elemento [2])==1 ):     
          lista_documentos_ya_evaluados_1.append(elemento)   

  lista_documentos_relevancia=lista_documentos_ya_evaluados_2 + lista_documentos_ya_evaluados_1 # haciendo esto, tenemos en una unica lista primero los documentos de rel 2 y depsues los de rel 1
  posicion=1
  for elemento in lista_documentos_relevancia:                  # recuperamos la lista de los documentos ordenador por relevancia y aplicamos la formula
    if (corte>0):
      if (posicion==1):                                 
        sumatorio+=(int(elemento[2]))                           # Se calcula el cociente entre la relevancia del documentos y el logaritmo de la posicion donde se encuentre
      else:
          sumatorio+=((int(elemento[2]))/math.log(posicion,2))  # Se calcula el cociente entre la relevancia del documentos y el logaritmo de la posicion donde se encuentra
      corte-=1
      posicion+=1

  return sumatorio

In [193]:
media_nDCG_corte10=0
media_nDCG_corte100=0
media_nDCG_corte10_E = 0
media_nDCG_corte100_E = 0
nDCG_corte10 = []
nDCG_corte100 = []
print("nDCG en corte 10 Sin modulo")
for i in range(1, len(cos_tfidf) + 1):
    nDCG_10 = discounted_acumulated_gain(Extract_values(10, lista_topics(i, Final_List_Relevancia)))/ideal_discounted_acumulated_gain(i, 10)
    nDCG_corte10.append(nDCG_10)
    print("TOPIC: ", i, ":", '{:.4f}'.format(nDCG_10))

print("nDCG en corte 100 Sin modulo")
for i in range(1, len(cos_tfidf) + 1):
    nDCG_100 = discounted_acumulated_gain(Extract_values(100, lista_topics(i, Final_List_Relevancia)))/ideal_discounted_acumulated_gain(i, 100)
    nDCG_corte100.append(nDCG_100)
    print("Topic", i, ":", '{:.4f}'.format(nDCG_100))

for i in range (1,len(cos_tfidf)+1):
  media_nDCG_corte10+= discounted_acumulated_gain(Extract_values(10,lista_topics(i,Final_List_Relevancia)))/ideal_discounted_acumulated_gain(i,10)    
  media_nDCG_corte100+= discounted_acumulated_gain(Extract_values(100,lista_topics(i,Final_List_Relevancia)))/ideal_discounted_acumulated_gain(i,100) 
  media_nDCG_corte10_E+= discounted_acumulated_gain(Extract_values(10,lista_topics(i,Final_List_Relevancia_Extended)))/ideal_discounted_acumulated_gain(i,10)    
  media_nDCG_corte100_E+= discounted_acumulated_gain(Extract_values(100,lista_topics(i,Final_List_Relevancia_Extended)))/ideal_discounted_acumulated_gain(i,100)     

media_nDCG_corte10=media_nDCG_corte10/len (cos_tfidf)
media_nDCG_corte100=media_nDCG_corte100/len (cos_tfidf)
media_nDCG_corte10_E=media_nDCG_corte10_E/len (cos_tfidf)
media_nDCG_corte100_E=media_nDCG_corte100_E/len (cos_tfidf)
print("- Media nDCG de la consulta sin expandir en corte 10:  ", '{:.4f}'.format(media_nDCG_corte10))
print("- Media nDCG de la consulta sin expandir en corte 100: ", '{:.4f}'.format(media_nDCG_corte100))
print("- Media nDCG de la consulta sin expandir en corte 10 Extendido:  ", '{:.4f}'.format(media_nDCG_corte10_E))
print("- Media nDCG de la consulta sin expandir en corte 100 Extendido: ", '{:.4f}'.format(media_nDCG_corte100_E))

nDCG en corte 10 Sin modulo
TOPIC:  1 : 0.4364
TOPIC:  2 : 0.3336
TOPIC:  3 : 0.3823
TOPIC:  4 : 0.3492
TOPIC:  5 : 0.2499
TOPIC:  6 : 0.2264
TOPIC:  7 : 0.5036
TOPIC:  8 : 0.3688
TOPIC:  9 : 0.5635
TOPIC:  10 : 0.1869
TOPIC:  11 : 0.0000
TOPIC:  12 : 0.4946
TOPIC:  13 : 0.6787
TOPIC:  14 : 0.3298
TOPIC:  15 : 0.3401
TOPIC:  16 : 0.5146
TOPIC:  17 : 0.3629
TOPIC:  18 : 0.3163
TOPIC:  19 : 0.4398
TOPIC:  20 : 0.2476
nDCG en corte 100 Sin modulo
Topic 1 : 0.7152
Topic 2 : 0.5540
Topic 3 : 0.7342
Topic 4 : 0.6703
Topic 5 : 0.5924
Topic 6 : 0.6367
Topic 7 : 0.7944
Topic 8 : 0.7209
Topic 9 : 0.7934
Topic 10 : 0.6388
Topic 11 : 0.4877
Topic 12 : 0.8479
Topic 13 : 0.8690
Topic 14 : 0.7503
Topic 15 : 0.7380
Topic 16 : 0.8310
Topic 17 : 0.6955
Topic 18 : 0.7724
Topic 19 : 0.7715
Topic 20 : 0.5999
- Media nDCG de la consulta sin expandir en corte 10:   0.3663
- Media nDCG de la consulta sin expandir en corte 100:  0.7107
- Media nDCG de la consulta sin expandir en corte 10 Extendido:   0.3597
- 

# Medias sin modulo

In [194]:
print("- Media Precisión en corte 5: ",'{:.4f}'.format(precision_5_media))
print("- Media Precisión en corte 10: ",'{:.4f}'.format(precision_10_media))
print("- Media Exhaustividad en corte 5: ",'{:.4f}'.format(exhaustidivad_5_media))
print("- Media Exhaustividad en corte 10: ",'{:.4f}'.format(exhaustidivad_10_media))
print("- Media ValorF en corte 5: ",'{:.4f}'.format(ValorF_5))
print("- Media ValorF en corte 10: ",'{:.4f}'.format(ValorF_10))
print("- Media Reciprocal Rank 1: ",'{:.4f}'.format(recc_1))
print("- Media Reciprocal Rank 2: ",'{:.4f}'.format(recc_2))
print("- Media Average Precision@100: ",'{:.4f}'.format(sum(AP_100) / len(AP_100)))
print("- Media nDCG de la consulta sin expandir en corte 10: ",'{:.4f}'.format(media_nDCG_corte10))
print("- Media nDCG de la consulta sin expandir en corte 100: ",'{:.4f}'.format(media_nDCG_corte100))

- Media Precisión en corte 5:  0.5000
- Media Precisión en corte 10:  0.4800
- Media Exhaustividad en corte 5:  0.0524
- Media Exhaustividad en corte 10:  0.0945
- Media ValorF en corte 5:  0.0949
- Media ValorF en corte 10:  0.1580
- Media Reciprocal Rank 1:  0.7327
- Media Reciprocal Rank 2:  0.3922
- Media Average Precision@100:  0.5308
- Media nDCG de la consulta sin expandir en corte 10:  0.3663
- Media nDCG de la consulta sin expandir en corte 100:  0.7107


#Medias con el modulo activo: 


In [195]:
print("- Media Precisión en corte 5 expandida: ",'{:.4f}'.format(precision_5_media_E))
print("- Media Precisión en corte 10 expandida: ",'{:.4f}'.format(precision_10_media_E))
print("- Media Exhaustividad en corte 5 expandida: ",'{:.4f}'.format(exhaustidivad_5_media_E))
print("- Media Exhaustividad en corte 10 expandida: ",'{:.4f}'.format(exhaustidivad_10_media_E))
print("- Media ValorF en corte 5 expandida: ",'{:.4f}'.format(ValorF_5_E))
print("- Media ValorF en corte 10 expandida: ",'{:.4f}'.format(ValorF_10_E))
print("- Media Reciprocal Rank 1 expandida: ",'{:.4f}'.format( media_Rec_E))
print("- Media Reciprocal Rank 2 expandida: ",'{:.4f}'.format(media_Rec_2_E))
print("- Media Average Precision@100 expandida: ",'{:.4f}'.format(sum(AP_100_E) / len(AP_100_E)))
print("- Media nDCG de la consulta sin expandir en corte 10 Extendido:  ", '{:.4f}'.format(media_nDCG_corte10_E))
print("- Media nDCG de la consulta sin expandir en corte 100 Extendido: ", '{:.4f}'.format(media_nDCG_corte100_E))

- Media Precisión en corte 5 expandida:  0.5200
- Media Precisión en corte 10 expandida:  0.4750
- Media Exhaustividad en corte 5 expandida:  0.0550
- Media Exhaustividad en corte 10 expandida:  0.0938
- Media ValorF en corte 5 expandida:  0.0996
- Media ValorF en corte 10 expandida:  0.1567
- Media Reciprocal Rank 1 expandida:  0.6975
- Media Reciprocal Rank 2 expandida:  0.4006
- Media Average Precision@100 expandida:  0.5431
- Media nDCG de la consulta sin expandir en corte 10 Extendido:   0.3597
- Media nDCG de la consulta sin expandir en corte 100 Extendido:  0.7081


# Modulo Reconocimiento de entidades de consultas de usuario. 

Aqui, hacemos un contador de todas las veces que aparecen entidades en los documentos, para poder premiar más a unos que a otros. Esto tarda un poco más ya que evalua cada posible nombre de los casi 8mil en cada documento html. Cabe destacar que comparamos los nombres originales (sin pasarlos por un proceso de stemming) y El corpus original (tambien sin pasar por el proceso de stemming) 

Se ha decidido hacerlo  de esta forma por que si intentamos comparar palabra a palabra tokenizada este doble bucle tarda al rededor de 18 minutos en completarse. Mientras que si lo hacemos con todo el texto, tarda 6-7 minutos




In [196]:
Docs_Coincidencia_Entidades_Total = {}
for i,doc in enumerate(Lista_Content2): 
  Docs_Coincidencia_Entidades_Total[Name_Files_clean[i]] = 0 #Inicializamos cada posición
  for name in Nombres_total:
    if name in doc:
      Docs_Coincidencia_Entidades_Total[Name_Files_clean[i]] += 1
print(Docs_Coincidencia_Entidades_Total)


{'2010-83-007': 0, '2010-83-028': 24, '2010-83-027': 39, '2010-83-006': 21, '2010-83-001': 19, '2010-83-024': 31, '2010-83-017': 40, '2010-83-071': 90, '2010-83-077': 39, '2010-83-050': 49, '2010-83-076': 24, '2010-83-085': 88, '2010-83-092': 77, '2010-83-089': 40, '2010-83-069': 27, '2010-83-056': 151, '2010-83-052': 92, '2010-83-029': 131, '2010-83-030': 60, '2010-83-083': 200, '2010-69-058': 64, '2010-69-063': 65, '2010-69-080': 61, '2010-69-019': 61, '2010-69-084': 63, '2010-69-023': 50, '2010-69-077': 30, '2010-69-006': 0, '2010-69-000': 108, '2010-69-054': 77, '2010-69-010': 61, '2010-69-064': 49, '2010-69-059': 19, '2010-69-003': 15, '2010-69-018': 210, '2010-69-044': 183, '2010-69-050': 57, '2010-79-031': 0, '2010-79-095': 22, '2010-79-001': 21, '2010-79-027': 23, '2010-79-051': 13, '2010-79-086': 44, '2010-79-071': 23, '2010-79-048': 37, '2010-79-054': 5, '2010-79-050': 18, '2010-79-082': 23, '2010-79-049': 45, '2010-79-037': 647, '2010-79-093': 42, '2010-79-061': 19, '2010-79

Función que premia a los documentos con la función descrita arriba.

In [197]:
def PremiarDocumentos(array,RecuentoEntidades):
  for a in array:
     if a[0] in RecuentoEntidades:
         a[1] = premiarCos(RecuentoEntidades[a[0]],a[1])
  return array

Extraemos los valores que necesitemos

In [198]:
def Extract_values_2(num,array):
  Lista_valores_extraidos=[]
  for i in range(0,num):
      Lista_valores_extraidos.append(array[i])
  return Lista_valores_extraidos

Función print

In [199]:
def entity_recognition_print(query, is_entity_recognition_active, document_list):
    print("-----QUERY-----")
    print(query)
    print("-----MODE-----")
    recognition_mode = "Entity recognition activated" if is_entity_recognition_active else "Entity recognition NOT activated"
    print(recognition_mode)
    print("-----TOP 20 WITH ENTITY RECOGNITION-----")
    print()
    table = []
    for i, document in enumerate(document_list, 1):
        table.append([i, document[0], document[1]])
    headers = ["Rank", "Document Name", "Cosine Similarity"]
    print(tabulate(table, headers=headers))

In [200]:
#ListEntity= ['who','Who','Which','which','whom','Whom']
consulta = input("Enter query:")
Entity=False 
consulta_palabras = consulta.split()   #dividimos la consulta en palabras
c = stemList(consulta_palabras) #Hacemos stemming
consulta_stem=" ".join(c)#lo volvemos a juntar para la vectorización

matriz_tfidf = tfidf_vectorizer.fit_transform(corpus)
query_tfidf = tfidf_vectorizer.transform([consulta_stem])
cos_tfidf_Modulo=cosine_similarity(query_tfidf, matriz_tfidf) #Obtnemos los valores de coseno
Lista_cos_names=matchNameCos(cos_tfidf_Modulo,Name_Files_clean) #Obtenemos una lista con los nombres de los documentos y su valor coseno

#Le pasamos la lista de pronombres relativos
if MatchNombreConsuta(ListEntity,consulta) == True: #si la consulta activa el modulo
  Entity = True
  Lista_Documentos_Premiados=PremiarDocumentos(Lista_cos_names,Docs_Coincidencia_Entidades_Total) #premiamos a los documentos
  Lista_Documentos_Premiados_Ordenada = sorted(Lista_Documentos_Premiados, key=lambda x: x[1], reverse=True)#ordenamos la lista por coseno
  Top20 = Extract_values_2(20,Lista_Documentos_Premiados_Ordenada) #Extraemos el top 20
  entity_recognition_print(consulta,Entity,Top20) #le pasamos al print todo
else: #si la consulta no activa el modulo
  Lista_Documentos_No_Premiados_Ordenada = sorted(Lista_cos_names, key=lambda x: x[1], reverse=True)
  Top20 = Extract_values_2(20,Lista_Documentos_No_Premiados_Ordenada)
  entity_recognition_print(consulta,Entity,Top20)

KeyboardInterrupt: ignored